In [1]:
cd '/home/psadmin/work/DataCleansing'

/home/psadmin/work/DataCleansing


In [2]:
import re
import os
import logging
import datetime
import numpy as np
import pandas as pd
import dataset as ds
from cleansing.singleton import singleton
from cleansing.stat import Stat
from cleansing.util.util import Util
from cleansing.util.merge_master import MergeMaster
from cleansing.master.master_data import MasterData
from cleansing.master.monitor_spec import MasterDataMonitorSpec
from cleansing.master.system_code import MasterDataSystemCode
from cleansing.master.check_list import MasterDataCheckList
from cleansing.master.equipment_pc import MasterDataEquipmentPC


In [58]:
df = pd.read_excel('getconfig/check_sheet.xlsx',
                           '検査レポート',
                           2)
df = df.drop(df.index[0:8])
df.columns

Index(['No', 'ホスト名', 'ドメイン', '接続IP', 'OS名', 'アーキテクチャ', 'CPU数', 'MEM容量',
       'ディスク構成', 'ネットワーク構成', '管理LAN', '検査成績', '検査結果', '備考'],
      dtype='object')

In [25]:
df['ネットワーク構成'].str.split(',\s').tolist()

[['[lo:127.0.0.1/8', 'eth0:192.168.10.1/24', 'eth0:1:192.168.10.4/24]'],
 ['[lo:127.0.0.1/8', 'ens160:192.168.0.20/24', 'docker0:172.17.0.1/16]'],
 ['[192.168.0.24]']]

In [37]:
# カッコを取り除く
df['ips'] = df['ネットワーク構成'].str.extract('\[(?P<ips>.+)\]', expand=False)
df['ips']

8     lo:127.0.0.1/8, eth0:192.168.10.1/24, eth0:1:1...
9     lo:127.0.0.1/8, ens160:192.168.0.20/24, docker...
10                                         192.168.0.24
Name: ip_addresses, dtype: object

In [47]:
# カンマ区切りで分割して、複数行に展開
df1 = pd.DataFrame()
df1['ip'] = df.ip_addresses.str.split(',', expand=True).stack().str.strip().reset_index(level=1, drop=True)
df1

,ip
8,lo:127.0.0.1/8
8,eth0:192.168.10.1/24
8,eth0:1:192.168.10.4/24
9,lo:127.0.0.1/8
9,ens160:192.168.0.20/24
9,docker0:172.17.0.1/16
10,192.168.0.24


In [56]:
# 先頭、文末の文字列を削除
df1['ip'] = df1['ip'].replace('.+:','', regex=True)
df1['ip'] = df1['ip'].replace('\/.+', '', regex=True)

In [57]:
df.drop(['ip_addresses'], axis=1).join(df1).reset_index(drop=True)

,No,ホスト名,ドメイン,接続IP,OS名,アーキテクチャ,CPU数,MEM容量,ディスク構成,ネットワーク構成,管理LAN,検査成績,検査結果,備考,ip
0,1,ostrich,Linux,192.168.10.1,CentOS release,x86_64,1,1877.2,"[/boot:500M, /:46.5G, [SWAP]:3G]","[lo:127.0.0.1/8, eth0:192.168.10.1/24, eth0:1:...",No data,66.7 %,"6 / 18 Failed : [meminfo, mem_total, net_route...",,127.0.0.1
1,1,ostrich,Linux,192.168.10.1,CentOS release,x86_64,1,1877.2,"[/boot:500M, /:46.5G, [SWAP]:3G]","[lo:127.0.0.1/8, eth0:192.168.10.1/24, eth0:1:...",No data,66.7 %,"6 / 18 Failed : [meminfo, mem_total, net_route...",,192.168.10.1
2,1,ostrich,Linux,192.168.10.1,CentOS release,x86_64,1,1877.2,"[/boot:500M, /:46.5G, [SWAP]:3G]","[lo:127.0.0.1/8, eth0:192.168.10.1/24, eth0:1:...",No data,66.7 %,"6 / 18 Failed : [meminfo, mem_total, net_route...",,192.168.10.4
3,2,cent7,Linux,192.168.0.20,CentOS Linux release,x86_64,1,1839.97,"[/boot:500M, /:13.9G, [SWAP]:1.6G]","[lo:127.0.0.1/8, ens160:192.168.0.20/24, docke...",No data,100.0 %,,,127.0.0.1
4,2,cent7,Linux,192.168.0.20,CentOS Linux release,x86_64,1,1839.97,"[/boot:500M, /:13.9G, [SWAP]:1.6G]","[lo:127.0.0.1/8, ens160:192.168.0.20/24, docke...",No data,100.0 %,,,192.168.0.20
5,2,cent7,Linux,192.168.0.20,CentOS Linux release,x86_64,1,1839.97,"[/boot:500M, /:13.9G, [SWAP]:1.6G]","[lo:127.0.0.1/8, ens160:192.168.0.20/24, docke...",No data,100.0 %,,,172.17.0.1
6,3,win2012,Windows,192.168.0.24,Microsoft Windows Server 2012 R2 Standard 評価版,64 ビット,1,4193848,[C:40],[192.168.0.24],No data,61.5 %,"5 / 13 Failed : [vm, NumCpu, VMHost, filesyste...",,192.168.0.24
